# サポートベクター回帰 (Support Vector Regression, SVR)

# 概要
「サポートベクトル」を用いた「回帰」分析

# 特徴
* 機械学習の分野で最も広く使われているアルゴリズムの1つ。
* 特徴次元数が大きいデータでも高い精度が出る。
* 非線形なデータも扱うことができる。

# 詳細 パターン１
* サポートベクトルマシン
    * サポートベクトル＝決定境界から最も近い正のサンプルと負のサンプル
    * 正の超平面,負の超平面＝サポートベクトルを通る超平面
    * マージン(＝2つの超平面の間隔)2/|w|を最大化する
    * 関数L=|w|^2/2を最小化する問題に置き換える
* 直線で分けることが出来ない場合(1) ソフトマージン分類
    * 境界からはみ出ることを許可する＆ペナルティを与える
    * スラック変数ξ＝どれだけはみ出したかの度合
    * 関数L=|w|^2/2+C*(Σ_i ξ(i)) を最小化する問題に置き換える
    * 実際には、等価な双対問題を導出して2次計画法を用いて最小化問題を解く
* 直線で分けることが出来ない場合(2) 非線形SVM
    * 変数x→非線形写像Φ(x)でより高い次元に拡張した空間においてSVMを適用する
    * RBFカーネルは、実質無限次元への拡張
    * 表現力が高い（過学習しやすいとも言う）
* サポートベクトル回帰
    * 回帰の誤差がεを超えたときにペナルティを与える
    * ξやLを同様に考えてSVMを適用する

# 説明 パターン２
* 線形回帰 f(x)=x*b
* 非線形回帰 f(x)=Φ(x)*w+c Φ:非線形写像
* SVMとSVRの比較
    * SVM「マージンの逆数(＝重み)＋誤分類の数」最小化
    * SVR「重み＋誤差」の最小化
* SVRの誤差関数：h(y-f(x)) = 「誤差y-f(x)が+/-ε以下なら0、誤差が...以上なら、|誤差|-ε」
    ※ある程度の誤差(ε以下)は無視する
* スラック変数ξ,ξ*を導入して、以下のように変形
    誤差関数 h(y-f(x)) = ξ+ξ*
* 以下、最小化するときの定式化は省略
    未定乗数法や二次計画問題、カーネルトリックなどを使う
* サポートベクター = 誤差がε以上のサンプル
* パラメーターの意味
    * ε:誤差の許容範囲
    * C:モデルの複雑さ
    * γ：カーネル

# 説明 パターン３
* サポートベクターマシンとの類推よりも、線形回帰の一種であるリッジ回帰から考える
* リッジ回帰において「誤差の二乗和」としたのを、「『誤差の絶対値 -ε（誤差がε以下なら０）』の和」としたもの = 線形サポートベクトル回帰
* 線形サポートベクトル回帰 + カーネルトリック = 非線形も扱えるサポートベクトル回帰


# ハイパーパラメータの高速最適化
* SVRのハイパーパラメータ
    * C の候補: $2^{-5}, 2^{-4}, …, 2^{9}, 2^{10}$ (16通り)
    * ε (イプシロン) の候補: $2^{-10}, 2^{-9}, …, 2^{-1}, 2^{0}$ (11通り)
    * γ (ガンマ) の候補: $2^{-20}, 2^{-19}, …, 2^{9}, 2^{10}$ (31通り)
    * グリッドサーチだと$16 \times 11 \times 31=5456$回も必要
* 高速で最適化する方法
    1. γ = グラム行列の分散が最大になるもの、と仮定
    2. C = 目的変数yの平均＋/-標準偏差×3、あたりがいいので、データが標準化済みなら、C=3、と仮定
    3. 1と2の、γとCを用いて、εのみを最適化（ε決定）
    4. 1と3の、γとεを用いて、Cのみを最適化（C決定）
    5. 3と4の、εとCを用いて、γのみを最適化（γ決定）
    6. パラメータ１つずつ最適化するので、$11+16+31=56$回で済む
* グラム行列とは？
    * 前提：カーネルk(x,y)と、入力x1,x2,...,xnがある
    * カーネル値 k(xi,xj) を i,j 成分とする行列がグラム行列
* ハイパーパラメータの値の候補はどうして2のべき乗・累乗なのか？
    1. 探索範囲が広いので、ある程度荒くする
    2. $10^n$よりも細かい
    3. 細かく最適化しても、予測性能が悪いと意味がない

# 誤差が一定のところにサンプルが固まる現象
1. SVR (Support Vector Regression, サポートベクター回帰) で回帰モデルを構築したとき  
    yy-plot(実測値 vs. 推定値プロット)において、対角線から一定に離れたところにサンプルが固まる  
     つまり全く同じ誤差のサンプルがある、 ε チューブの境界に、サンプルが固まる、場合がある
2. この現象自体にはまったく問題はない
3. SVRの計算式からの証明。詳細は省く。  
    モデル構築における最適化の段階で、0 < αj,aj* < C となったサンプルは、誤差がε,-εになる  
4. 3から言えること
    a. ε チューブの内側のサンプル、つまり誤差が小さいサンプルは、SVR モデルに影響しないため、
      SVR モデルはノイズの影響を受けにくい
    b. ε チューブの外側のサンプル、つまり誤差が大きいサンプルは、  
      どんなに誤差が大きくても、SVR モデルへの影響を C でコントロールできるため、  
      SVR モデルは目的変数の外れ値の影響は受けにくい

## 自分の解釈
* 上記では、誤差がεになるサンプルがどんなものかを説明はしてるけど、そういうサンプルが多い(1.の現象)の説明はしてないような気がする
* サンプルが固まるのはεチューブの「境界」というより「内部」では？その場合は、以下の通り理解できる。
    * SVRは「誤差ε以上のサンプルをなるべく減らすようにモデルを構築する」
    * つまり「ε以下の誤差があっても気にしない(モデル構築には影響しない)」ので、そのモデルで予測した場合、誤差が0~εになるものが多いということになる


# 参考文献
* サポートベクター回帰(Support Vector Regression, SVR)～サンプル数10000以下ならこれを使うべし！～  
    https://datachemeng.com/supportvectorregression/
* 第9回 サポートベクターマシンで非線形な競馬予測に挑む　（AlphaImpact）  
    https://alphaimpact.jp/2017/03/16/svm/
* [Pythonコードあり] サポートベクター回帰(Support Vector Regression, SVR)のハイパーパラメータを高速に最適化する方法  
    https://datachemeng.com/fastoptsvrhyperparams/
* SVM・SVRのハイパーパラメータの値の候補はどうして2のべき乗・累乗なのか？  
    http://univprof.com/archives/16-06-25-4229971.html
* あなたはサポートベクターマシン・回帰 (Support Vector Machine or Regression, SVM or SVR)におけるハイパーパラメータの設定に時間がかかっていませんか？  
    http://univprof.com/archives/16-07-14-4701508.html
* SVR(サポートベクター回帰)で誤差が一定のところにサンプルが固まるのはどうして？何か問題があるの？ → SVR の特徴も確認！  
    https://datachemeng.com/svr_epsilon_tube/


